In [ ]:
![](https://storage.googleapis.com/kaggle-datasets-images/310/684/3503c6c827ca269cc00ffa66f2a9c207/dataset-cover.jpg)

# About Dataset
## Context
It is important that credit card companies are able to recognize fraudulent credit card transactions so that customers are not charged for items that they did not purchase.

# Content
The dataset contains transactions made by credit cards in September 2013 by European cardholders.
This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.

It contains only numerical input variables which are the result of a PCA transformation. Unfortunately, due to confidentiality issues, we cannot provide the original features and more background information about the data. Features V1, V2, … V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'. Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature 'Amount' is the transaction Amount, this feature can be used for example-dependant cost-sensitive learning. Feature 'Class' is the response variable and it takes value 1 in case of fraud and 0 otherwise.

Given the class imbalance ratio, we recommend measuring the accuracy using the Area Under the Precision-Recall Curve (AUPRC). Confusion matrix accuracy is not meaningful for unbalanced classification.

# Credit Card Fraud Detection

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df = pd.read_csv('/kaggle/input/creditcardfraud/creditcard.csv')
df.head()

In [ ]:
df.tail()

In [ ]:
df.shape

In [ ]:
df.info()


In [ ]:
df.describe()

In [ ]:
df.duplicated().sum()

In [ ]:
df.isna().sum()

In [ ]:
df['Class'].value_counts().plot(kind='bar')

In [ ]:
df['Class'].value_counts()

In [ ]:
df['Amount'].hist()

In [ ]:
df['Time'].hist()

# Preprocessing

In [ ]:
from imblearn.over_sampling import RandomOverSampler

In [ ]:
ROS = RandomOverSampler()

In [ ]:
target = 'Class'
X = df.drop(columns = target)
y = df[target]

In [ ]:
X, y = ROS.fit_resample(X,y)

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=5) 
X_train.shape,X_test.shape,y_train.shape,y_test.shape

# Model

In [ ]:
from xgboost import XGBClassifier
model = XGBClassifier()
model.fit(X_train,y_train)

In [ ]:
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

# Evaluation

In [ ]:
from sklearn.metrics import accuracy_score,recall_score,precision_score,classification_report, ConfusionMatrixDisplay

In [ ]:
# Train
print(f'accuracy_score = {accuracy_score(y_train,y_pred_train)}')
print(f'recall_score = {recall_score(y_train,y_pred_train)}')
print(f'precision_score = {precision_score(y_train,y_pred_train)}')
print(f'classification_report = {classification_report(y_train,y_pred_train)}')

In [ ]:
# Test 
print(f'accuracy_score = {accuracy_score(y_test,y_pred_test)}')
print(f'recall_score = {recall_score(y_test,y_pred_test)}')
print(f'precision_score = {precision_score(y_test,y_pred_test)}')
print(f'classification_report = {classification_report(y_test,y_pred_test)}')

In [ ]:
ConfusionMatrixDisplay.from_predictions(y_test,y_pred_test)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

feature_importances = model.feature_importances_ 
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feature_importance_df)
plt.title('Model Feature Importances') 
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.show()

In [ ]:
X.columns

In [ ]:
def pred_fun(Time, V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,model):
    input= [[Time, V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount]]
    pred = model.predict(input)
    return pred

In [ ]:
pred_fun(Time=406, V1=-2.31227,V2=1.951992,V3=-1.609,V4=3.9979,V5=-0.522188,V6=-1.426545,V7=-2.537387,V8= 1.391657,V9=-2.770089,V10=-2.772272,V11= 3.202033, V12= -2.899907,V13=-0.595222,V14=-3.289254,V15=-2.389724,V16=-4.140747,V17= -2.830056,V18=-0.016822,V19=0.416956,V20= 0.126911,V21=0.517232,V22=-0.035049, V23=-0.465211,V24=0.320198,V25=0.044519,V26=0.177840,V27=0.261145,V28=-0.143276,Amount=0 ,model=model)

In [ ]:
df.iloc[541]